import and install

In [4]:
import pyomo.environ as pyo

!apt-get install -y glpk-utils



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


Model

In [5]:
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 6)

# Parameters
demand = {1:2, 2:2, 3:3, 4:0, 5:3, 6:1}

Cp = 4     # production capacity
Cs = 2     # storage capacity
f  = 100   # setup cost
c  = 200   # unit production cost
h  = 50    # holding cost
S0 = 1     # initial inventory

# Variables
model.x = pyo.Var(model.T, domain=pyo.NonNegativeIntegers)
model.S = pyo.Var(model.T, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.T, domain=pyo.Binary)

# Objective
def obj_rule(m):
    return sum(f*m.y[t] + c*m.x[t] + h*m.S[t] for t in m.T)

model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)



Constraints

In [6]:

def inventory_rule(m, t):
    if t == 1:
        return m.S[t] == S0 + m.x[t] - demand[t]
    return m.S[t] == m.S[t-1] + m.x[t] - demand[t]

model.Inventory = pyo.Constraint(model.T, rule=inventory_rule)

def production_capacity_rule(m, t):
    return m.x[t] <= Cp * m.y[t]

model.ProdCap = pyo.Constraint(model.T, rule=production_capacity_rule)

def storage_capacity_rule(m, t):
    return m.S[t] <= Cs

model.StoreCap = pyo.Constraint(model.T, rule=storage_capacity_rule)

# Ending inventory
model.EndInv = pyo.Constraint(expr=model.S[6] == 0)



Solve

In [7]:
solver = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
solver.solve(model)

# ------------------
# Results
# ------------------
for t in model.T:
    print(f"Month {t}: Production={model.x[t].value}, "
          f"Inventory={model.S[t].value}, Setup={model.y[t].value}")

print("Total Cost =", pyo.value(model.Obj))

Month 1: Production=1.0, Inventory=0.0, Setup=1.0
Month 2: Production=2.0, Inventory=0.0, Setup=1.0
Month 3: Production=3.0, Inventory=0.0, Setup=1.0
Month 4: Production=0.0, Inventory=0.0, Setup=0.0
Month 5: Production=4.0, Inventory=1.0, Setup=1.0
Month 6: Production=0.0, Inventory=0.0, Setup=0.0
Total Cost = 2450.0
